In [2]:
from bs4 import BeautifulSoup

In [3]:
from nltk.stem.wordnet import WordNetLemmatizer

In [4]:
wn_lemmatizer = WordNetLemmatizer()

In [14]:
with open('../electronics/positive.review') as fid:
    positive_review_html = fid.read()
positive_reviews = BeautifulSoup(positive_review_html).findAll('review_text')

In [15]:
len(positive_reviews)

1000

In [16]:
with open('../electronics/negative.review') as fid:
    negative_review_html = fid.read()
negative_reviews = BeautifulSoup(negative_review_html).findAll('review_text')

In [17]:
len(negative_reviews)

1000

In [20]:
import nltk

In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
all_texts = [review.text.strip().lower() for review in positive_reviews]
all_texts += [review.text.strip().lower() for review in negative_reviews]
labels = ['positive' for _ in positive_reviews]
labels += ['negative' for _ in negative_reviews]

In [25]:
for i, text in enumerate(all_texts):
    words = nltk.tokenize.word_tokenize(text)
    text = ' '.join(map(lambda w: wn_lemmatizer.lemmatize(w), words))
    all_texts[i] = text

In [26]:
all_texts[0]

'i purchased this unit due to frequent blackout in my area and 2 power supply going bad . it will run my cable modem , router , pc , and lcd monitor for 5 minute . this is more than enough time to save work and shut down . equally important , i know that my electronics are receiving clean power . i feel that this investment is minor compared to the loss of valuable data or the failure of equipment due to a power spike or an irregular power supply . a always , amazon had it to me in < 2 business day'

In [30]:
with open('../stopwords.txt') as fid:
    stop_words = set(map(lambda w: w.strip(), fid.readlines()))

In [31]:
cnt_vectorizer = CountVectorizer(stop_words=stop_words)
X = cnt_vectorizer.fit_transform(all_texts)

In [32]:
from sklearn.preprocessing import LabelEncoder

In [33]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

In [34]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [35]:
from sklearn.model_selection import train_test_split

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [82]:
model = LogisticRegression(C=0.1, max_iter=200)

In [83]:
model.fit(X_train, y_train)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [84]:
model.score(X_test, y_test)

0.79

In [85]:
model.score(X_train, y_train)

0.9635714285714285

In [86]:
from sklearn.metrics import classification_report

In [87]:
y_hat = model.predict(X)

In [88]:
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

           0       0.93      0.90      0.91      1000
           1       0.90      0.93      0.91      1000

    accuracy                           0.91      2000
   macro avg       0.91      0.91      0.91      2000
weighted avg       0.91      0.91      0.91      2000

